# Magics in Jupyter

## Intro & background

## Context - common workflows in Jupyter

### Audience participation - what are the common reasons why you leave a Jupyter notebook?

* Data ingestion or filtering (sed, awk)
* Database operations
* Unit testing
* Frontend development (browser dev tools)

## Problem - we have users who only see the notebooks

## Solution - magics

### What are magics?

#### Cell magics vs line magics

#### Built into kernel but also can be created

## %sh

## %sql

## ihtml

Using magics to work with iFrames inside Jupyter notebooks

## Making your own magics

Behind the scenes on how these are created

## Conclusion and next steps

